In [3]:
from sklearn.decomposition import NMF
from numpy import *
import scipy.io
import matplotlib.pyplot as plt

import os

folder = "C:\\Users\\Dori\\Dropbox (Brain Energy Lab)\\Everything\\dori\\data\\BehaviourOnWheel\\Sawk\\20190404"
#folder = "C:\\Users\\Dori\\Dropbox (Brain Energy Lab)\\Everything\\dori\\data\\BehaviourOnWheel\\Sawk\\20190304"
#folder = "C:\\Users\\Dori\\Documents\\OIST\\20190218_09_38_05_R1spont\\cellsort\\"
#traces = scipy.io.loadmat(os.path.join(folder,'suite2p\\spikes.mat'))['traces']
traces = scipy.io.loadmat("".join((folder,'\\', 'suite2p\\traces.mat')))['traces']
#traces = traces-traces.min(axis=1,keepdims=True)
X = traces
X[X<0] = 0
traces.shape


Locomotion = scipy.io.loadmat("".join((folder,'\\','locomotion.mat')))['loco']
vel = np.diff(Locomotion, axis=0)
vel = np.insert(vel, 0, 0)
vel[vel<-0.2/7.51] = 0
vel[vel>0.5/7.51] = 0


model = NMF(n_components=3, init='random', random_state=0)
W = model.fit_transform(X)
H = model.components_

plt.figure()
plt.plot(H[0,:].T,)#color=colors[i] )
plt.xlabel('Frames')
plt.ylabel('Component 1 weight')

plt.figure()
plt.plot(H[1,:].T,)#color=colors[i] )
plt.xlabel('Frames')
plt.ylabel('Component 1 weight')

plt.figure()
plt.plot(H[2,:].T,)#color=colors[i] )
plt.xlabel('Frames')
plt.ylabel('Component 1 weight')


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\locomotion.mat'

Some of the things wrong with the things wrong with the code above and how to fix it:
* All imports should be at the top of the file.
* Don't import *, replace with import numpy as np.
* Needs to be put into a function, there's no reason to save all the cell traces if you run this for multiple folders, it will take up too much memory
* Variable names could be more descriptive. What does X mean? 
* Variable names generally shouldn't be capitalized (Locomotion -> locomotion)
* Don't hardcode folders or filenames. One way to change this is to have folder as input to the function. Even better is to get the user inputting the folder using a dialog window, such as askdirectory from the Tkinter package.
* The code is a bit erratic in terms of whitespaces, only use whitespace when there is a separate paragraph
* Random comments that are not very useful, only use comments when they say something about the code!
* The number 7.51 and both of the thresholds (-0.2 and 0.5) are hardcoded. Have these as optional inputs, or calculate the thresholds from the data. The number of components is also hardcoded
* Duplicate code for plotting, just use a forloop

In [6]:
def calculate_nmf(folder, trace_file, loco_file, fps=7.51, lower_thresh = -0.2, upper_thresh = 0.5, n_components=3):
    ''' Calculates the non-negative matrix factorization for a dataset that is in folder
    
    Keyword arguments:
    folder - folder path containing locomotion and data file
    trace_file - path to data matrix, relative to folder
    loco_file - path to locomotion data, relative to folder
    fps - frames per second of locomotion recording (default = 7.51)
    lower_thresh - Lower speed cutoff (default = -0.2)
    upper_thresh - Upper speed cutoff (default = 0.5)
    n_components - The number of components to calculate in the non-negative matrix factorization (default = 3)
    
    Output arguments
    W - The weight of each cell in the different components from the NMF
    H - The time series for each components from the NMF
    '''
    cell_fluorescence = scipy.io.loadmat(os.path.join(folder,trace_file))['traces']
    cell_fluorescence[cell_fluorescence<0] = 0
    locomotion = scipy.io.loadmat(os.path.join(folder,'\\','locomotion.mat'))['loco']
    
    velocity = np.diff(locomotion, axis=0)
    velocity = np.insert(velocity, 0, 0)
    velocity[velocity<lower_thresh/fps] = 0
    velocity[velocity>upper_thresh/fps] = 0

    model = NMF(n_components=3, init='random', random_state=0)
    W = model.fit_transform(cell_fluorescence)
    H = model.components_
    
    for i,h in enumerate(H):
        plt.figure()
        plt.plot(h.T,)
        plt.xlabel('Frames')
        plt.ylabel(f'Component {i} weight')
        plt.show()
        
    return W, H
   
    